In [ ]:
import numpy as np
import pandas as pd
import string
import random
from tqdm import tqdm
from typing import *

import torch

import pytorch_lightning as pl

from loguru import logger

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
from src.experiments.experiments import get_experiment

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
experiment = get_experiment('resnet')

In [ ]:
!pip install torchmetrics --quiet

In [ ]:
import torchmetrics
import glob
import pytorch_lightning.metrics as pl_metrics
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from sklearn.metrics import (
    roc_auc_score,
    confusion_matrix,
    precision_score,
    recall_score,
    accuracy_score,
)

In [ ]:
import pickle

with open('all_results.pickle', 'rb') as handle:
    all_results = pickle.load(handle)

In [ ]:
# weights_to_test = [weight for weight in glob.glob('../mlruns/2/**/artifacts/weights/*.ckpt') if weight not in all_results]

# for model_path in weights_to_test:
    
#     trainer = pl.Trainer(gpus=1, precision=16)

#     experiment = experiment.load_from_checkpoint(
#     #     '../2_1635799472.ckpt',
#         model_path,
#         train_data_path='../.data/medical_data_train_with_metadata',
#         val_data_path='../.data/medical_data_val_with_metadata',
#         learning_rate= 0.003,
#         batch_size= 512,
#         sequence_length= 16384,
#         metrics = {
#             "stats": torchmetrics.StatScores(reduce='macro', num_classes=2 ),
#             "confusion": torchmetrics.ConfusionMatrix(normalize='true', num_classes=2 ),
#             "auroc": torchmetrics.AUROC(average='macro', num_classes=2 ),
#             "f1_score": torchmetrics.F1(average='macro', num_classes=2 ),
#             "precision": torchmetrics.Precision(average='macro', num_classes=2 ),
#             "recall": torchmetrics.Recall(average='macro', num_classes=2 ),
#             "accuracy": torchmetrics.Accuracy(average='macro', num_classes=2 ),
#         },
#         balanced_dataloaders=False,
#         strict=False

#     )
    
#     def test_step(batch, batch_idx) -> None:
#         _, y, y_hat, loss, patient_id = experiment.step(batch, stage="test")

#         experiment.log(
#             f"test_loss", loss.item()
#         )

#         y_prob = torch.softmax(y_hat, 1).cpu()
#         y_hat = y_hat.cpu()
#         y_pred = torch.argmax(y_hat, dim=1).cpu()
#         y = y.cpu()

#     #     experiment.results["confusion"].append(confusion_matrix(y_true=y, y_pred=y_pred, normalize='true').ravel())
#     #     experiment.results["precision"].append(precision_score(y_true=y, y_pred=y_pred))
#     #     experiment.results["recall"].append(recall_score(y_true=y, y_pred=y_pred))
#     #     experiment.results["accuracy"].append(accuracy_score(y_true=y, y_pred=y_pred))

#         experiment.results["make_auroc_chart"].append({'y_true': y, 'y_prob': y_prob, 'y_pred': y_pred, 'patient_id': patient_id})

#     experiment.results = defaultdict(list)
#     experiment.test_step = test_step
    
#     test_results = trainer.test(
#         experiment, datamodule=experiment.data_module
#     )
    
#     all_results[model_path] = {
#         'y_true': torch.stack([item for sublist in experiment.results['make_auroc_chart'] for item in sublist['y_true']]),
#         'y_probas' : torch.stack([item for sublist in experiment.results['make_auroc_chart'] for item in sublist['y_prob']]),
#         'patient_ids' : torch.stack([item for sublist in experiment.results['make_auroc_chart'] for item in sublist['patient_id']]),
    
#     }
    
# with open('all_results.pickle', 'wb') as handle:
#     pickle.dump(all_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
!pip install scikit-plot --quiet

In [ ]:
from sklearn import metrics

In [ ]:
all_model_results = []
for model_key, results in all_results.items():
    model_key_id = model_key.split('/')[3]
    
    
    target = results['y_true']
    preds = torch.argmax(results['y_probas'], dim=1)
    patient_ids = results['patient_ids']
    
#     print(model_key_id)
#     print(metrics.classification_report(target, preds))

    auc_fpr, auc_tpr, auc_thresholds = metrics.roc_curve(target, results['y_probas'][:, -1])
    
    optimal_idx = np.argmax(auc_tpr - auc_fpr)
    optimal_threshold = auc_thresholds[optimal_idx]
    print(optimal_threshold)
    
    def cutoff_youdens_j(fpr,tpr,thresholds):
        j_scores = tpr-fpr
        j_ordered = sorted(zip(j_scores,thresholds))
        return j_ordered[-1][1]

    print(cutoff_youdens_j(auc_fpr, auc_tpr, auc_thresholds))
    
    data = pd.DataFrame(
        zip(target.numpy(), results['y_probas'][:, -1].numpy(), patient_ids.cpu().numpy()), 
        columns = ['true_label', f'confidence_in_being_hypertensive', 'patient_id']
    )
    data['model_no'] = f'{model_key_id}'
    
    all_model_results.append(data)    
    break

In [ ]:
plt.figure(figsize=(7,7))
lw = 2
best_threshold = metrics.auc(auc_fpr, auc_tpr)
plt.plot(auc_fpr, auc_tpr, color='darkorange', lw=lw, label=f'ROC curve (area = {best_threshold:0.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
i = np.arange(len(auc_tpr)) # index for df
roc = pd.DataFrame(
    {
        'fpr' : pd.Series(auc_fpr, index=i),
        'tpr' : pd.Series(auc_tpr, index = i), 
        '1-fpr' : pd.Series(1-auc_fpr, index = i), 
        'tf' : pd.Series(auc_tpr - (1-auc_fpr), index = i), 
        'thresholds' : pd.Series(auc_thresholds, index = i)
    }
)

# Plot tpr vs 1-fpr
fig, ax = plt.subplots(figsize=(10,6))
_ = plt.plot(roc['tpr'], label='sensitivity/tpr')
_ = plt.plot(roc['1-fpr'], color = 'red', label='specificity/1-fpr')
_ = plt.xlabel('1-False Positive Rate')
_ = plt.ylabel('True Positive Rate')
_ = plt.title('Receiver operating characteristic')
_ = ax.set_xticklabels([])
_ = plt.legend()

# print(roc.iloc[(roc.tf).abs().argsort()[:1]])

In [ ]:
import math
# calculate the g-mean for each threshold
gmeans = np.sqrt(auc_tpr * (1-auc_fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (auc_thresholds[ix], gmeans[ix]))

tuned_fpr = auc_fpr[ix] 
tuned_tpr = auc_tpr[ix]
tuned_threshold = auc_thresholds[ix]

tuned_sensitivity = tuned_tpr / (tuned_tpr + tuned_fpr)
tuned_tnr = 1-tuned_fpr
tuned_specificity = tuned_tnr/(tuned_tnr+tuned_fpr)
tuned_fnr = 1 - tuned_tpr

print(f"""

non_hypertensive_pateints = {125262}
hypertensive_patients = {11714}

TP = {tuned_tpr:0.3f},   FP = {tuned_fpr:0.3f}
FN = {tuned_fnr:0.3f},  TN = {tuned_tnr:0.3f}

Of those patients with hypertension, 
  number of patients classified as hypertensive = tp = {tuned_tpr:0.3f}
  
Of those patients without hypertension, 
  number of patients classified as not hypertensive = tn = {tuned_tnr:0.3f}

sensitivity = tp/(tp+fn) = {tuned_sensitivity*100:0.2f}%

specificity = tn/(tn+fp) = {tuned_specificity*100:0.2f}%

""")

In [ ]:
# Manually defining the threshold

In [ ]:
given_value = 0.5
a_list = auc_thresholds.tolist()
absolute_difference_function = lambda list_value : abs(list_value - given_value)

closest_value = min(a_list, key=absolute_difference_function)
closest_index = a_list.index(closest_value)
print(closest_value, closest_index)

In [ ]:
test_tpr = auc_tpr[closest_index]
test_sensitivity = test_tpr
test_sensitivity
f"sensitivity = {test_sensitivity*100:0.2f}%"

In [ ]:
test_fpr = auc_fpr[closest_index] 
test_specificity = 1-test_fpr
f"specificity = {test_specificity*100:0.2f}%"